# 1.5 专用 Notebook：DAG 画图 & DoWhy 自动识别调整集（入门强化版）

本 Notebook 配套于**1.5 因果图（DAG）与变量选择**，包含两部分：
1）用 `networkx` 画出示例 DAG；2）用 `DoWhy` 自动识别背门调整集（MSAS）。

> 你可以把本 Notebook 作为项目模板：把自己的 DAG 粘进来，立刻得到**最小充分调整集**候选。

## 0. 环境准备（如已安装可跳过）

> 如未安装依赖，请先在本地环境中运行：
```bash
pip install -U networkx matplotlib dowhy pydot
```
> *说明*：`DoWhy` 可直接解析我们在代码里写的 DOT 图字符串；`pydot` 用于图解析。

In [ ]:
# 导入与版本检查
import sys, platform, importlib

def check(pkg):
    try:
        m = importlib.import_module(pkg)
        return f"{pkg}: {getattr(m,'__version__','(no __version__)')}"
    except Exception as e:
        return f"{pkg}: NOT FOUND ({e})"

print('Python:', sys.version)
print('OS:', platform.platform())
for pkg in ['networkx','matplotlib','dowhy','pydot']:
    print(check(pkg))


## 1. 用 `networkx` 画出示例 DAG

示例结构（与讲义一致）：
- `Age → Drug; Age → BP`
- `Severity → Drug; Severity → BP`（治疗前严重度为混杂）
- `Drug → BP`（治疗对结局的直接效应）
- `Drug → BP_post → BP`（`BP_post` 为**治疗后**指标/中介）

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

nodes = ['Age','Severity','Drug','BP','BP_post']
edges = [
    ('Age','Drug'), ('Age','BP'),
    ('Severity','Drug'), ('Severity','BP'),
    ('Drug','BP'),
    ('Drug','BP_post'), ('BP_post','BP')
]
G = nx.DiGraph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

plt.figure(figsize=(7,5))
pos = nx.spring_layout(G, seed=2025)
nx.draw(G, pos, with_labels=True, node_size=2000, arrowsize=20)
plt.title('DAG: Drug -> BP with Confounders & Mediator')
plt.show()


## 2. 用 `DoWhy` 自动识别“最小充分调整集”（MSAS）

> 这里仅做**识别**演示。数据用“空壳”`DataFrame` 以声明变量名即可。

In [ ]:
import pandas as pd
from dowhy import CausalModel

cols = ['Age','Severity','Drug','BP','BP_post']
df = pd.DataFrame({c: [] for c in cols})

graph = r'''
digraph {
Age -> Drug; Age -> BP;
Severity -> Drug; Severity -> BP;
Drug -> BP;
Drug -> BP_post; BP_post -> BP;
}
'''

model = CausalModel(data=df, treatment='Drug', outcome='BP', graph=graph)
identified = model.identify_effect()
print('=== Identified Estimand ===')
print(identified)

def extract_backdoor_set(identified):
    """Extract candidate backdoor adjustment sets from a DoWhy identified estimand.
    Returns a dict with 'candidates', 'union', and 'raw' string."""
    backdoor_info = identified.estimands.get('backdoor')
    sets = []
    exprs = backdoor_info if isinstance(backdoor_info, list) else [backdoor_info]
    for expr in exprs:
        text = str(expr)
        import re
        matches = re.findall(r"\{([A-Za-z0-9_,\s]+)\}", text)
        for m in matches:
            s = [x.strip() for x in m.split(',') if x.strip()]
            if s:
                sets.append(set(s))
    union = sorted(set().union(*sets)) if sets else []
    return {'candidates': [sorted(s) for s in sets], 'union': union, 'raw': str(backdoor_info)}

info = extract_backdoor_set(identified)
print('候选背门调整集（解析）：', info['candidates'] if info['candidates'] else '(未能解析，见 raw 表达式)')
print('变量并集（便于直接赋值给清单）：', info['union'])
print('\nRaw 表达式：\n', info['raw'])


## 3. 套用自己的 DAG（模板）

把下面的 `graph` 换成你的 DAG（DOT 语法），并设置 `treatment` 与 `outcome`，即可自动识别调整集。

In [ ]:
# === 修改以下三个量 ===
YOUR_TREATMENT = 'X'
YOUR_OUTCOME   = 'Y'
your_cols = ['X','Y','Z1','Z2','M','A']  # 用你的变量名替换
your_graph = r'''
digraph {
# 示例：请替换为你的 DAG
Z1 -> X; Z1 -> Y;
Z2 -> X; Z2 -> Y;
X -> Y;
X -> M; M -> Y;   # 中介示意
X -> A; U -> A;   # 碰撞示意（U 未观测，仅示意，不放入数据列）
}
'''

your_df = pd.DataFrame({c: [] for c in your_cols})
your_model = CausalModel(data=your_df, treatment=YOUR_TREATMENT, outcome=YOUR_OUTCOME, graph=your_graph)
your_id = your_model.identify_effect()
print(your_id)

print('\n=== 你的背门调整集（解析） ===')
print(extract_backdoor_set(your_id))


## 4. 从 MSAS 到代码变量清单（项目模板）

- **PS 模型**：只放**治疗前混杂**（MSAS）+ 少量建模必需基线变量；
- **结果模型（DR/TMLE 等）**：可放相同治疗前变量 + 合理的非线性或交互（仍然是治疗前信息）；
- **明确排除**：治疗后变量/中介、碰撞及其后代。

In [ ]:
# 假设从上面的 info['union'] 得到 MSAS 候选：
pre_covs = info['union']
print('最终用于 PS/背门调整的治疗前变量清单：', pre_covs)
nonlinear_terms = []  # 例如 ['Age^2', 'Age*Severity'] —— 建模时再具体实现
print('结果模型额外项（示意）：', nonlinear_terms)
